In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 


master_url = "spark://localhost:7077"
driver_host_ip = "host.docker.internal"

In [ ]:
import os, pathlib


HADOOP_HOME = r"jupyter_local/.hadoop/hadoop-3.3.6"
driver_java_opt = fr"-Dhadoop.home.dir={HADOOP_HOME}"
os.environ["HADOOP_HOME"] = HADOOP_HOME
os.environ["PATH"] = str(pathlib.Path(HADOOP_HOME) / "bin") + ";" + os.environ.get("PATH","")

In [ ]:
spark = (
    SparkSession.builder.appName("LocalDriverToDockerCluster")
    .master(master_url)
    .config("spark.driver.host", driver_host_ip)
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.port", "7078")
    .config("spark.blockManager.port", "7079")
    .config("spark.executor.memory", "1G")
    .config("spark.executor.cores", "1")
    .config("spark.executor.instances", "2")
    # --- Cấu hình MinIO (S3A) ---
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9990")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .config("spark.hadoop.fs.s3a.access.key", "minio")
    .config("spark.hadoop.fs.s3a.secret.key", "minio123")
    .getOrCreate()
)

In [ ]:
df = spark.read.format("json").load("s3a://corin/test.json")